In [2]:
# -*- coding: utf-8 -*-

## standard packages
import numpy as np
import os
import datetime

## Config Parster for initiation
import configparser

## HTTP scraping packages
from lxml import html
import requests

#import httplib2
import urllib

from bs4 import BeautifulSoup, SoupStrainer
import json

## Database
import sqlite3

import src.utils as utils

In [3]:
pagesConfig = configparser.ConfigParser()
pagesConfig.read("config/siteSpecifics.ini")

for section in pagesConfig.sections():
    print(section)
    #for option in pagesConfig.options(section):
        #print(option, pagesConfig.get(section, option))
    #    print (section, option)

WebPages
additionalAdInfo
dbFilepath
dbInit


In [4]:
searchPage = pagesConfig.get('WebPages', 'mobile.de')
print(searchPage)

http://suchen.mobile.de/fahrzeuge/search.html?scopeId=C&damageUnrepaired=NO_DAMAGE_UNREPAIRED&scopeId=C&isSearchRequest=true&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&pageNumber=


In [28]:
#payload = {'scopeId': ['C', 'C'],
#           'damageUnrepaired': 'NO_DAMAGE_UNREPAIRED',
#           'isSearchRequest': 'true',
#           'sortOption.sortBy': 'creationTime',
#           'sortOption.sortOrder': 'DESCENDING'}

def getAdIdList(searchPage, priceStep=1000):
    payload=dict()
    dataIDs = np.zeros([0, 4])

    for factor in range(5):
        maxPrice = (factor+1)*priceStep
        minPrice = factor*priceStep

        payload['maxPrice'] = maxPrice
        payload['minPrice'] = minPrice
        for i in range(50):
            #print(i)
            payload['pageNumber'] = i+1

            req = requests.get(searchPage, params=payload)
            #print(req.url)
            soup = BeautifulSoup(req.text, "lxml")
            for link in soup.find_all('a'):
                if link.has_attr('data-ad-id'):
                    #print(link['data-ad-id'])
                    if link['data-ad-id'] in dataIDs[:, 0]:
                        #print(link['data-ad-id'], "already existing")
                        pass
                    else:
                        onPos = link.get_text('href').find('online seit ')
                        onlineSince = link.get_text('href')[onPos+12 : onPos+22]

                        dataIDs = np.vstack([dataIDs, [link['data-ad-id'], link['href'], link.get_text('href'), onlineSince]])

        print(factor, minPrice, maxPrice, dataIDs.shape)
        
    return dataIDs
        
        
dataIDs = getAdIdList(searchPage)
print(len(dataIDs))

C:\Users\Florian\Anaconda2\envs\ipykernel_py3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


0 0 1000 (1001, 4)
1 1000 2000 (1970, 4)
2 2000 3000 (2937, 4)
3 3000 4000 (3906, 4)
4 4000 5000 (4874, 4)
4874


In [30]:
print(dataIDs[0, 0])
print(dataIDs[0])

248088793
['248088793'
 'https://suchen.mobile.de/fahrzeuge/details.html?id=248088793&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&maxPrice=1000&minPrice=0&pageNumber=1&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&fnai=prev'
 ' href5hrefNeuhrefVolkswagen Caddy SDI 9K LKWhrefInserat online seit 28.07.2017 14:48href399 €hrefEZ 07/2000, 249.327\xa0km, 47\xa0kW\xa0(64\xa0PS)hrefVan / Minibus, hrefUnfallfreihref, Diesel, Schaltgetriebe, HU 11/2017, 2/3 Türenhref≈\u20095,9\u2009l/100km (komb.), ≈\u2009159\u2009g CO₂/km (komb.)hrefDE-98663 Westhausen, PrivatanbieterhrefFinanzierunghref, hrefVersicherunghrefGeparkthrefParkenhrefKontakt'
 '28.07.2017']


In [32]:
allInfos = utils.getInfoFromPage(dataIDs[0, 1])

In [36]:
allInfos['description']

'ich hatte ihn während meines Bauvorhabens, am Caddy funktioniert alles, aber der Rost hat ihn stark zugesetzt. deshalb an Bastler. Rest TÜV bis Oktober 2017. \r\nübrigens: http://www.bussgeld-info.de/bussgeldkatalog-tuev-papiere/\r\n2-4 Monate ohne TÜV 15€; 4-8 Monate 25€, ab 8 gibts Punkte ;-)\r\nAch und nur nebenbei für die Sparfüchse: oben steht Verbrauch 5,9l/100km, der fährt bei mir mit etwa 5,2 Liter\r\nDer Caddy ist angemeldet und kann mit meiner Zulassung gegen 100€ Kaution überführt werden.\r\nIch bin zurzeit in Elternzeit und daher fast immer Daheim. einfach anrufen'

In [37]:
allInfos['firstSeen'] = dataIDs[0][3]
allInfos['lastSeen'] = str(datetime.datetime.now().date)
allInfos

{'ad': {'condition': 'USED',
  'currency': 'EUR',
  'price': '399',
  'specifics': {'make': 'Volkswagen', 'model': 'Caddy'}},
 'adCondition': 'USED',
 'adCurrency': 'EUR',
 'adFirstRegMonth': '7',
 'adFirstRegYear': '2000',
 'adId': 248088793,
 'adImage': 'https://i.ebayimg.com/00/s/OTAwWDE2MDA=/z/0N8AAOSwrk5Zey9u/$_23.JPG',
 'adImageBase64': 'aHR0cHM6Ly9pLmViYXlpbWcuY29tLzAwL3MvT1RBd1dERTJNREE9L3ovME44QUFPU3dyazVaZXk5dS8k',
 'adPrice': '399',
 'adSegment': 'Car',
 'adSpecificsFuel': 'DIESEL',
 'adSpecificsMake': 'Volkswagen',
 'adSpecificsMakeId': 25200,
 'adSpecificsMakeModel': 'Volkswagen Caddy',
 'adSpecificsMakeModelBase64': 'Vm9sa3N3YWdlbiBDYWRkeQ==',
 'adSpecificsModel': 'Caddy',
 'adSpecificsModelDescription': 'Caddy SDI 9K LKW',
 'adSpecificsModelId': 9,
 'airbag': 'No Information',
 'category': 'Van / Minibus',
 'climatisation': 'No Information',
 'countryVersion': 'No Information',
 'cubicCapacity': '1.896\xa0cm³',
 'dealerCity': 'Westhausen',
 'dealerCountry': 'DE',
 'deale

## initiate Database

In [ ]:
#connection = sqlite3.connect('src/dataBase/carStats.db')
#cursor = connection.cursor()

# delete 
#cursor.execute("""DROP TABLE car;""")

#cursor.execute(sql_create_command)

# never forget this, if you want the changes to be saved:
#connection.commit()

#connection.close()

In [69]:
%pwd

'H:\\GitHub_Projects\\UsedCarStats'

In [90]:
from importlib import reload
reload(utils)

<module 'src.utils' from 'H:\\GitHub_Projects\\UsedCarStats\\src\\utils.py'>

In [91]:
database.close()
database = utils.DataBase('src/dataBase/carStats.db')

In [92]:
database.connect()

In [93]:
database.getTableNames()

[('car',)]

In [94]:
database.execute("""SELECT * FROM car""")

[]

In [95]:
database.deleteAllContents()

In [96]:
cursor = database.getCursor()
cursor.description

In [97]:
cursor.execute("""DROP TABLE car;""")


In [98]:
attributes = dict()
for option in pagesConfig.options('dbInit'):
    attributes[option] = pagesConfig.get('dbInit', option)
database.createTable(attributes)
cursor = database.getCursor()
cursor.description

In [99]:
database.getTableNames()

[('car',)]

In [100]:
database.execute("""SELECT * FROM car""")

[]

In [101]:
dataIDs.shape

(4874, 4)

In [102]:
def insertIntoDB(dictionary, superKey=''):
    columns = ''
    values = ''
    for key, val in dictionary.items():
        if type(val) == dict:
            c, v = insertIntoDB(val, superKey=superKey + key)
            columns += c
            values += v
        elif key.lower() in attributes.keys():
            columns += superKey + key + ', '
            values += '"' + str(val) + '"' + ', '

    
    return columns, values

for i in range(dataIDs.shape[0]):
    try:
        allInfos = utils.getInfoFromPage(dataIDs[i, 1])
        allInfos['firstSeen'] = dataIDs[i][3]
        allInfos['lastSeen'] = str(datetime.datetime.now().date)

        columns, values = insertIntoDB(allInfos)

        database.execute('''INSERT INTO car ( %s ) VALUES ( %s );''' %(columns[:-2], values[:-2]) )
    except Exception as e:
        print (i)
        print (e)

database.execute("""SELECT * FROM car""")

20
Expecting value: line 1 column 2 (char 1)
62
near "Klimaautomatik": syntax error
97
Expecting value: line 1 column 2 (char 1)
105
near "7": syntax error
124
Expecting value: line 1 column 2 (char 1)
130
Expecting value: line 1 column 2 (char 1)
239
Expecting value: line 1 column 2 (char 1)
270
Expecting value: line 1 column 2 (char 1)
303
Expecting value: line 1 column 2 (char 1)
346
Expecting value: line 1 column 2 (char 1)
365
Expecting value: line 1 column 2 (char 1)
453
Expecting value: line 1 column 2 (char 1)
456
Expecting value: line 1 column 2 (char 1)
462
near "90657": syntax error
533
Expecting value: line 1 column 2 (char 1)
535
Expecting value: line 1 column 2 (char 1)
538
Expecting value: line 1 column 2 (char 1)
548
Expecting value: line 1 column 2 (char 1)
647
near "A": syntax error
649
Expecting value: line 1 column 2 (char 1)
769
near "Klima": syntax error
801
Expecting value: line 1 column 2 (char 1)
846
Expecting value: line 1 column 2 (char 1)
888
Expecting value

KeyboardInterrupt: 

In [104]:
result = database.execute("""SELECT * FROM car""")
len(result)

1421

In [105]:
database.save()
database.close()

In [88]:
i = 42
allInfos = utils.getInfoFromPage(dataIDs[i, 1])
allInfos['firstSeen'] = dataIDs[i][3]
allInfos['lastSeen'] = str(datetime.datetime.now().date)
allInfos

{'ad': {'condition': 'USED',
  'currency': 'EUR',
  'price': '990',
  'specifics': {'make': 'Opel', 'model': 'Astra'}},
 'adCondition': 'USED',
 'adCurrency': 'EUR',
 'adFirstRegMonth': '3',
 'adFirstRegYear': '1996',
 'adId': 248086929,
 'adImage': 'https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/s0QAAOSwv89ZezGz/$_23.JPG',
 'adImageBase64': 'aHR0cHM6Ly9pLmViYXlpbWcuY29tLzAwL3MvTVRJd01GZ3hOakF3L3ovczBRQUFPU3d2ODlaZXpHei8k',
 'adPrice': '990',
 'adSegment': 'Car',
 'adSpecificsFuel': 'PETROL',
 'adSpecificsHsn': '35',
 'adSpecificsMake': 'Opel',
 'adSpecificsMakeId': 19000,
 'adSpecificsMakeModel': 'Opel Astra',
 'adSpecificsMakeModelBase64': 'T3BlbCBBc3RyYQ==',
 'adSpecificsModel': 'Astra',
 'adSpecificsModelDescription': 'Astra Caravan 1,6 GL SSD, TÜV 07/2019',
 'adSpecificsModelId': 5,
 'adSpecificsTsn': '321',
 'airbag': 'Front-Airbags',
 'category': 'Kombi',
 'climatisation': 'No Information',
 'countryVersion': 'No Information',
 'cubicCapacity': '1.598\xa0cm³',
 'dealerCity': 'Asperg/

In [86]:
print(dataIDs[i, 1])

https://suchen.mobile.de/fahrzeuge/details.html?id=248088646&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&maxPrice=1000&minPrice=0&pageNumber=1&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING


In [78]:
cursor = database.getCursor()
cursor.description

(('adid', None, None, None, None, None, None),
 ('adcondition', None, None, None, None, None, None),
 ('adcurrency', None, None, None, None, None, None),
 ('adfirstregmonth', None, None, None, None, None, None),
 ('adfirstregyear', None, None, None, None, None, None),
 ('adprice', None, None, None, None, None, None),
 ('adsegment', None, None, None, None, None, None),
 ('adspecificsfuel', None, None, None, None, None, None),
 ('adspecificshsn', None, None, None, None, None, None),
 ('adspecificsmake', None, None, None, None, None, None),
 ('adspecificsmakeid', None, None, None, None, None, None),
 ('adspecificsmakemodel', None, None, None, None, None, None),
 ('adspecificsmodel', None, None, None, None, None, None),
 ('adspecificsmodeldescription', None, None, None, None, None, None),
 ('adspecificsmodelid', None, None, None, None, None, None),
 ('dealercity', None, None, None, None, None, None),
 ('dealercountry', None, None, None, None, None, None),
 ('dealerzip', None, None, None, N

In [226]:
for i in cursor.description:
    print (i[0])

adid
adcondition
adcurrency
adfirstregmonth
adfirstregyear
adprice
adsegment
adspecificsfuel
adspecificshsn
adspecificsmake
adspecificsmakeid
adspecificsmakemodel
adspecificsmodel
adspecificsmodeldescription
adspecificsmodelid
dealercity
dealercountry
dealerzip
featurevariants
isdealer
sellertype
usersipaddress
category
countryversion
mileage
cubiccapacity
power
fuel
fuelconsumption
emission
numseats
doorcount
transmission
emissionssticker
firstregistration
numberofpreviousowners
hu
climatisation
parkassist
airbag
manufacturercolorname
color
interior
features
description
firstseen
lastseen


In [108]:
connection.close()